## Imported libraries and function

In [2]:
import pandas as pd
from sklearn import datasets, linear_model
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
import timeit
from sklearn.svm import SVR
def get_train_data():
    data = pd.read_csv('train.csv')

    data["pickup_datetime"] = pd.to_datetime(data["pickup_datetime"])
    data["dropoff_datetime"] = pd.to_datetime(data["dropoff_datetime"])
    data['pick_year'] = data["pickup_datetime"].dt.year
    data['pick_month'] = data["pickup_datetime"].dt.month
    data['pick_day'] = data["pickup_datetime"].dt.day

    data['pickup_year'] = data["pickup_datetime"].dt.year
    data['pickup_month'] = data["pickup_datetime"].dt.month
    data['pickup_day'] = data["pickup_datetime"].dt.day

    data['dropoff_year'] = data["dropoff_datetime"].dt.year
    data['dropoff_month'] = data["dropoff_datetime"].dt.month
    data['dropoff_day'] = data["dropoff_datetime"].dt.day

    pickup_hour =  data["pickup_datetime"].dt.hour
    pickup_minute = data["pickup_datetime"].dt.minute
    pickup_second = data["pickup_datetime"].dt.second

    dropoff_hour =  data["dropoff_datetime"].dt.hour
    dropoff_minute = data["dropoff_datetime"].dt.minute
    dropoff_second = data["dropoff_datetime"].dt.second

    pickup_time = pickup_hour*3600+pickup_minute*60+pickup_second
    data['pickup_time'] = pickup_time

    dropoff_time = dropoff_hour*3600+dropoff_minute*60+dropoff_second
    data['dropoff_time'] = dropoff_time

    data = data.drop(['pick_month'],axis=1)
    data =data.drop(['pick_day'],axis=1)
    data =data.drop(['pick_year'],axis=1)
    
    return data

def get_test_data():
    data = pd.read_csv('test.csv')

    data["pickup_datetime"] = pd.to_datetime(data["pickup_datetime"])
    data['pick_year'] = data["pickup_datetime"].dt.year
    data['pick_month'] = data["pickup_datetime"].dt.month
    data['pick_day'] = data["pickup_datetime"].dt.day

    data['pickup_year'] = data["pickup_datetime"].dt.year
    data['pickup_month'] = data["pickup_datetime"].dt.month
    data['pickup_day'] = data["pickup_datetime"].dt.day

    pickup_hour =  data["pickup_datetime"].dt.hour
    pickup_minute = data["pickup_datetime"].dt.minute
    pickup_second = data["pickup_datetime"].dt.second


    pickup_time = pickup_hour*3600+pickup_minute*60+pickup_second
    data['pickup_time'] = pickup_time

    data = data.drop(['pick_month'],axis=1)
    data =data.drop(['pick_day'],axis=1)
    data =data.drop(['pick_year'],axis=1)
    
    return data

from math import sin, cos, sqrt, atan2, radians

# approximate radius of earth in km
def get_L2(lat1,lon1,lat2,lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


def getXY_L2(train_x,train_y,test_x):
    x = train_x.copy()
    y = train_y.copy()
    test = test_x.copy()

    df_lat1 = np.array(x['pickup_latitude'])
    df_lon1 = np.array(x['pickup_longitude'])
    df_lat2 = np.array(x['dropoff_latitude'])
    df_lon2 = np.array(x['dropoff_longitude'])

    dft_lat1 = np.array(test['pickup_latitude'])
    dft_lon1 = np.array(test['pickup_longitude'])
    dft_lat2 = np.array(test['dropoff_latitude'])
    dft_lon2 = np.array(test['dropoff_longitude'])

    L2 = []
    for idx in range(len(df_lat1)):
        lat1 = df_lat1[idx]
        lon1 = df_lon1[idx]
        lat2 = df_lat2[idx]
        lon2 = df_lon2[idx]
        L2.append(get_L2(lat1,lon1,lat2,lon2))
    L2_t = []
    for idx in range(len(dft_lat1)):
        lat1 = dft_lat1[idx]
        lon1 = dft_lon1[idx]
        lat2 = dft_lat2[idx]
        lon2 = dft_lon2[idx]
        L2_t.append(get_L2(lat1,lon1,lat2,lon2))
    x['L2'] = L2
    test['L2'] = L2_t
    
    return x,y,test
    
    
    
def get_poly(degree,x,test):
    poly = PolynomialFeatures(degree=degree)
    x_ = poly.fit_transform(x)
    test_ = poly.fit_transform(test)
    
    return x_,test_
    
def get_csv(filename,pred_y):
    duration_content = np.rint(pred_y)
    id_content = np.array(test_id)

    df = pd.DataFrame({
        'id':id_content,
        'trip_duration':duration_content
    })

    # Create the csv file
    fn = filename + '.csv'
    df.to_csv(fn,index= False)

# Now, upload the csv file to
# https://www.kaggle.com/c/nyc-taxi-trip-duration/submit
    

# Dataframe for LR
### Create Standard x and y

In [ ]:
# Read data frame
df_train = get_train_data()
df_test = get_test_data()

# Drop id, vendor_id, dropoff_month, dropoff_day, dropoff_time, store_and_fwd_flag
df_train= df_train.drop(['id'],axis=1)
df_train= df_train.drop(['vendor_id'],axis=1)
df_train= df_train.drop(['dropoff_month'],axis=1)
df_train= df_train.drop(['dropoff_day'],axis=1)
df_train= df_train.drop(['dropoff_year'],axis=1)
df_train= df_train.drop(['dropoff_time'],axis=1)
df_train= df_train.drop(['store_and_fwd_flag'],axis=1)
df_train= df_train.drop(['pickup_datetime'],axis=1)
df_train= df_train.drop(['dropoff_datetime'],axis=1)

test_id = df_test['id']
df_test= df_test.drop(['id'],axis=1)
df_test= df_test.drop(['vendor_id'],axis=1)
df_test= df_test.drop(['store_and_fwd_flag'],axis=1)
df_test= df_test.drop(['pickup_datetime'],axis=1)


# Make standard x,y
train_y = df_train['trip_duration']
train_x = df_train.drop(['trip_duration'],axis=1)


test_x = df_test
#test_id


### Basic - train
0.86725, 1156

In [15]:
regr = linear_model.LinearRegression()


# Actual training
start_time = timeit.default_timer()
regr.fit(train_x, train_y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = regr.predict(test_x)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('basic_LR',pred_y)



fit time is: 0.3887655229991651
predict time is: 0.026782438999362057


### L2 - train
0.68746, 1070

In [18]:
x,y,test = getXY_L2(train_x,train_y,test_x)


regr = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr.fit(x, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = regr.predict(test)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('L2_LR',pred_y)


fit time is: 0.5603255240021099
predict time is: 0.026782438999362057


### L1 - train
0.69480, 1085

In [19]:
x = train_x.copy()
y = train_y.copy()
test = test_x.copy()

df_lat1 = np.array(x['pickup_latitude'])
df_lon1 = np.array(x['pickup_longitude'])
df_lat2 = np.array(x['dropoff_latitude'])
df_lon2 = np.array(x['dropoff_longitude'])

dft_lat1 = np.array(test['pickup_latitude'])
dft_lon1 = np.array(test['pickup_longitude'])
dft_lat2 = np.array(test['dropoff_latitude'])
dft_lon2 = np.array(test['dropoff_longitude'])

L1 = []
for idx in range(len(df_lat1)):
    lat1 = df_lat1[idx]
    lon1 = df_lon1[idx]
    lat2 = df_lat2[idx]
    lon2 = df_lon2[idx]
    L1.append(abs(lat1-lat2)+abs(lon1-lon2))
L1_t = []
for idx in range(len(dft_lat1)):
    lat1 = dft_lat1[idx]
    lon1 = dft_lon1[idx]
    lat2 = dft_lat2[idx]
    lon2 = dft_lon2[idx]
    L1_t.append(abs(lat1-lat2)+abs(lon1-lon2))
x['L1'] = L1
test['L1'] = L1_t

regr = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr.fit(x, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = regr.predict(test)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('L1_LR',pred_y)


fit time is: 0.5812664789991686
predict time is: 0.026782438999362057


### Polynomial(degree2) with L2 - train
0.63158, 1014

In [21]:
x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(2,x,test)

regr = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr.fit(x_, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = regr.predict(test_)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('poly_2_LR_L2',pred_y)






fit time is: 9.723537067999132
predict time is: 0.026782438999362057


### Polynomial(degree3) with L2 - train
0.60280, 974

In [22]:
x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(3,x,test)

regr = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr.fit(x_, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = regr.predict(test_)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('poly_3_LR_L2',pred_y)






fit time is: 71.19835357400007
predict time is: 0.026782438999362057


## Polynomial(degree4) with L2 - train
not run yet

In [23]:
x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(4,x,test)

regr = linear_model.LinearRegression()

# Actual training
start_time = timeit.default_timer()
regr.fit(x_, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = regr.predict(test_)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('poly_3_LR_L2',pred_y)



#Not run yet


KeyboardInterrupt: 

### Lasso_D2 with L2 -Train
not run yet

In [4]:

x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(2,x,test)

regr = linear_model.LinearRegression()


cv_list = [0.001, 0.01,0.1,0.5,5,10]
score_list = []
for cv in cv_list:
    clf = linear_model.Lasso(alpha=cv)
    scores = cross_val_score(clf, x_, y, cv=5)
    score_list.append(scores.mean())
max_idx = np.argmax(np.array(score_list))
print("best alpha is {}".format(cv_list[max_idx]))
clf = linear_model.Lasso(alpha=cv_list[max_idx])

# Actual training
start_time = timeit.default_timer()
clf.fit(x_, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = clf.predict(test_)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('Lasso_D2_L2',pred_y)

/Users/wayne/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/wayne/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/wayne/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/wayne/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not co

KeyboardInterrupt: 

### Lasso(degree 3) with L2 -Train

In [14]:

x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(3,x,test)

regr = linear_model.LinearRegression()


cv_list = [0.001,0.1,1,10]
score_list = []
for cv in cv_list:
    clf = linear_model.Lasso(alpha=cv)
    scores = cross_val_score(clf, x_, y, cv=5)
    score_list.append(scores.mean())
max_idx = np.argmax(np.array(score_list))
print("best alpha is {}".format(cv_list[max_idx]))
clf = linear_model.Lasso(alpha=cv_list[max_idx])

# Actual training
start_time = timeit.default_timer()
clf.fit(x_, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = clf.predict(test_)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('Lasso_D3_L2',pred_y)

/Users/wayne/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


1553.5645713300037


### Rige(degree 2) with L2 -Train

In [ ]:
x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(2,x,test)

regr = linear_model.LinearRegression()


cv_list = [0.001, 0.01,0.1,0.5,5,10]
score_list = []
for cv in cv_list:
    clf = Ridge(alpha=cv)
    scores = cross_val_score(clf, x_, y, cv=5)
    score_list.append(scores.mean())
max_idx = np.argmax(np.array(score_list))
print("best alpha is {}".format(cv_list[max_idx]))
clf = Ridge(alpha=cv_list[max_idx])

# Actual training
start_time = timeit.default_timer()
clf.fit(x_, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = clf.predict(test_)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('Ridge_D2_L2',pred_y)

### Neural Network Regression


In [27]:
x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(2,x,test)

layer_list = [2,10,100]

from sklearn.neural_network import MLPRegressor

for layer in layer_list:
    

    clf = MLPRegressor(hidden_layer_sizes=(layer,),  activation='relu', solver='adam',    alpha=0.001,batch_size='auto',
               learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
               random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
               nesterovs_momentum=True, early_stopping=True, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
               epsilon=1e-08)


    # Actual training
    start_time = timeit.default_timer()
    clf.fit(x_, y)
    fit_elapsed = timeit.default_timer() - start_time

    #start_time = timeit.default_timer()
    pred_y = clf.predict(test_)
    #predict_elapsed = timeit.default_timer() - start_time


    print ("fit time is: {}".format(fit_elapsed))
    print ("predict time is: {}".format(predict_elapsed))

    fn = 'MLP_L2_' + str(layer)
    get_csv(fn,pred_y)

59.79876787198009


### SVR 

In [ ]:
x,y,test = getXY_L2(train_x,train_y,test_x)

x_,test_ = get_poly(2,x,test)

clf = SVR(C=1.0, epsilon=0.2)


# Actual training
start_time = timeit.default_timer()
clf.fit(x, y)
fit_elapsed = timeit.default_timer() - start_time

#start_time = timeit.default_timer()
pred_y = clf.predict(test)
#predict_elapsed = timeit.default_timer() - start_time


print ("fit time is: {}".format(fit_elapsed))
print ("predict time is: {}".format(predict_elapsed))


get_csv('SVR',pred_y)